#  read data

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from tqdm.notebook import tqdm
from copy import deepcopy
import time
import networkx as nx
import os
import sys
import matplotlib.pyplot as plt
#os.environ['PROJ_LIB'] = os.path.dirname(sys.argv[0])

/anaconda3/envs/pytorch/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string
PROJ: proj_create_from_database: SQLite error on SELECT name, type, coordinate_system_auth_name, coordinate_system_code, datum_auth_name, datum_code, area_of_use_auth_name, area_of_use_code, text_definition, deprecated FROM geodetic_crs WHERE auth_name = ? AND code = ?: no such column: area_of_use_auth_name


In [2]:
# 55.6s
#file_path = 'data/blink_data/BLinkDataPt2_2005.csv'
file_path = 'data/blink_data/BLinkDataPt2_2005.csv'

col_names = ['c{0:02d}'.format(i) for i in range(32) ]
col_names = ['mesh_code', 'map_level','area_code', 'link_id', 'diff',
             'road_type_code','lane_num_code', 
             'start_trans_id', 'end_tras_id',
             'link_length', 'link_type_code',
             'oneway_limit_flag','stop_way_limit_flag',
             'start_lon','start_lat', 'latlng_diff_num'] + col_names
df = pd.read_csv(file_path, encoding='shift_jis', names=col_names, skiprows=3)
df = df[['mesh_code', 'map_level','area_code', 'link_id', 'diff',
             'road_type_code','lane_num_code', 
             'start_trans_id', 'end_tras_id',
             'link_length', 'link_type_code',
             'oneway_limit_flag','stop_way_limit_flag',
             'start_lon','start_lat', 'latlng_diff_num']]
df

,mesh_code,map_level,area_code,link_id,diff,road_type_code,lane_num_code,start_trans_id,end_tras_id,link_length,link_type_code,oneway_limit_flag,stop_way_limit_flag,start_lon,start_lat,latlng_diff_num
0,362257,0,13,65536,8,8,3,1096,1112,24,1,0,0,56650585,11265378,1
1,362257,0,13,65545,16,8,3,1112,1114,62,1,0,0,56650578,11265279,2
2,362257,0,13,65562,64,8,3,1114,1116,352,1,0,0,56650652,11265031,5
3,362257,0,13,65627,73,8,3,1116,9766118,425,1,0,0,56652019,11264286,8
4,362257,0,13,65701,31,8,3,9766118,1004,174,1,0,0,56653925,11264443,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12789531,684210,0,31,20748759,368,8194,3,9266167,9266168,2306,1,0,0,65439886,20938592,70
12789532,684210,0,31,20749128,161,8194,3,9266168,9432046,963,1,0,0,65446175,20932085,25
12789533,684210,0,31,20749290,8,8194,3,9432046,9266757,46,1,0,0,65448893,20928994,1
12789534,684210,0,31,20749299,248,8194,3,9266757,9266722,1531,1,0,0,65449107,20928874,25


In [3]:
df.dtypes

mesh_code              int64
map_level              int64
area_code              int64
link_id                int64
diff                   int64
road_type_code         int64
lane_num_code          int64
start_trans_id         int64
end_tras_id            int64
link_length            int64
link_type_code         int64
oneway_limit_flag      int64
stop_way_limit_flag    int64
start_lon              int64
start_lat              int64
latlng_diff_num        int64
dtype: object

In [4]:
print('Unique area codes: ', df.area_code.unique())
print('No. of unique road types: ', df.road_type_code.nunique())
print('No. of unique link_ids: ', df.link_id.nunique())
print('No. of duplicated link_ids:', df.link_id.duplicated().sum())

Unique area codes:  [13 16 17 19 18 21 22 23 26 27 31]
No. of unique road types:  32
No. of unique link_ids:  12286507
No. of duplicated link_ids: 503029


In [5]:
blink = pd.read_csv('data/blink_data/BlinkData.csv', encoding='shift_jis')
blink = blink[blink.roadname.notnull()]
blink.columns = ['area_code', 'link_id', 'diff', 'roadname', 'routenumber',
       'roadtype', 'link_type_code', 'tollroad', 'motorway', 'hwy', 'oneway_limit_flag',
       'speedlimit', 'hwyupdown']
blink

,area_code,link_id,diff,roadname,routenumber,roadtype,link_type_code,tollroad,motorway,hwy,oneway_limit_flag,speedlimit,hwyupdown
8269,17,2674410,16,名城バイパス,331.0,2,1,0,0,0,0,40,0
8270,17,2674427,8,名城バイパス,331.0,2,1,0,0,0,0,40,0
8271,17,2674436,40,名城バイパス,331.0,2,1,0,0,0,0,40,0
8272,17,2674477,8,名城バイパス,331.0,2,1,0,0,0,0,40,0
8273,17,2674486,24,名城バイパス,331.0,2,1,0,0,0,0,40,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3513216,31,23564775,64,札樽自動車道,NaN,0,16,1,1,1,1,0,0
3513379,31,23594976,8,釧標国道,272.0,2,1,0,0,0,0,0,0
3513382,31,23595284,48,釧標国道,272.0,2,1,0,0,0,0,50,0
3513383,31,23595333,224,釧標国道,272.0,2,1,0,0,0,0,50,0


In [ ]:
# for i,roadname in enumerate(blink.roadname.unique()):
#     print(i," ",roadname)

In [6]:
print('Unique Area Codes:', blink.area_code.unique())
print('No. of unique roadnames:', blink.roadname.nunique())
print('No. of unique route numbers:', blink.routenumber.nunique())
print('No. of unique link_ids:', blink.link_id.nunique())
print('No. of duplicated link_ids:', blink.link_id.duplicated().sum())
print('No. of blink link_id not present in the main df:', len(blink) - blink.link_id.isin(df.link_id).sum())

Unique Area Codes: [17 21 22 23 26 27 31]
No. of unique roadnames: 3403
No. of unique route numbers: 583
No. of unique link_ids: 334509
No. of duplicated link_ids: 257
No. of blink link_id not present in the main df: 206


In [7]:
#Merging of both dataframes - df and blink (on common columns)
blinkmerged = pd.merge(blink, df[['mesh_code', 'area_code', 'link_id', 'diff', 'road_type_code', 'lane_num_code', 
                                  'start_trans_id', 'end_tras_id', 'link_length', 'link_type_code', 'start_lon', 'start_lat']], 
                       how='left', on=['area_code', 'link_id', 'diff'])

In [8]:
#Check 10s
tmp = [1 for x,y in (zip(blinkmerged[blinkmerged.mesh_code.isna()].link_id.values, 
                        blinkmerged[blinkmerged.mesh_code.isna()].area_code.values)) if x not in df.link_id.values] 
                              #if y not in df[df.link_id==x].link_type_code.values]
sum(tmp)

206

In [9]:
blinkmerged.isna().sum()

area_code                 0
link_id                   0
diff                      0
roadname                  0
routenumber          126042
roadtype                  0
link_type_code_x          0
tollroad                  0
motorway                  0
hwy                       0
oneway_limit_flag         0
speedlimit                0
hwyupdown                 0
mesh_code               773
road_type_code          773
lane_num_code           773
start_trans_id          773
end_tras_id             773
link_length             773
link_type_code_y        773
start_lon               773
start_lat               773
dtype: int64

In [10]:
# len=25
capital_road_list =[
    "首都高速湾岸線",
    "首都高速神奈川３号狩場線",
    "首都高速神奈川２号三ツ沢線",
    "首都高速神奈川１号横羽線",
    "首都高速１号羽田線",
    "首都高速神奈川５号大黒線",
    "首都高速神奈川６号川崎線",
    "首都高速３号渋谷線",
    "首都高速２号目黒線",
    "首都高速４号新宿線",
    "首都高速中央環状線",
    "首都高速都心環状線",
    "首都高速１号上野線",
    "首都高速１１号台場線",
    "首都高速１０号晴海線",
    "首都高速９号深川線",
    "首都高速５号池袋線",
    "首都高速埼玉大宮線",
    "首都高速八重洲線",
    "首都高速６号向島線",
    "首都高速７号小松川線",
    "首都高速６号三郷線",
    "首都高速川口線",
    "首都高速埼玉新都心線",
    "首都高速神奈川７号横浜北線"
]

In [11]:
blinkmerged_capital=blinkmerged.loc[blinkmerged["roadname"].isin(capital_road_list)]
blinkmerged_capital

,area_code,link_id,diff,roadname,routenumber,roadtype,link_type_code_x,tollroad,motorway,hwy,...,hwyupdown,mesh_code,road_type_code,lane_num_code,start_trans_id,end_tras_id,link_length,link_type_code_y,start_lon,start_lat
174019,27,83585420,64,首都高速湾岸線,NaN,1,16,1,1,1,...,0,533904.0,40961.0,3.0,4752850.0,4752852.0,352.0,16.0,64336501.0,16306747.0
174020,27,83585485,24,首都高速湾岸線,NaN,1,16,1,1,1,...,0,533904.0,40961.0,3.0,4752852.0,4760412.0,108.0,16.0,64337721.0,16305681.0
174021,27,83585609,48,首都高速湾岸線,NaN,1,16,1,1,1,...,0,533904.0,40961.0,3.0,4752853.0,4752854.0,252.0,16.0,64336863.0,16312838.0
174022,27,83585658,48,首都高速湾岸線,NaN,1,16,1,1,1,...,0,533904.0,40961.0,3.0,4752854.0,4752856.0,253.0,16.0,64336290.0,16311905.0
174023,27,83585806,56,首都高速湾岸線,NaN,1,16,1,1,1,...,0,533904.0,40961.0,3.0,4752858.0,4752859.0,312.0,16.0,64336901.0,16313005.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277877,27,118415091,8,首都高速５号池袋線,NaN,1,16,1,1,1,...,0,533955.0,40961.0,3.0,9851262.0,9851263.0,37.0,16.0,64364195.0,16485615.0
277878,27,118415100,56,首都高速５号池袋線,NaN,1,16,1,1,1,...,0,533955.0,40961.0,3.0,9851263.0,6905077.0,349.0,16.0,64364356.0,16485535.0
278691,27,118770346,64,首都高速中央環状線,NaN,1,4,1,1,1,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
278692,27,118770411,56,首都高速中央環状線,NaN,1,20,1,1,1,...,0,533936.0,40961.0,4.0,9870120.0,9434909.0,302.0,20.0,64399186.0,16405002.0


In [12]:
### restrict the lat lon to  and mmlatitude <= 35.90 and mmlatitude >= 35.36  and mmlongitude <= 139.947   and mmlongitude >= 139.537
blinkmerged_capital_new= blinkmerged_capital[blinkmerged_capital['start_lon'].between(139.537*128*3600, 139.947*128*3600, inclusive=True)& blinkmerged_capital['start_lat'].between(35.36*128*3600, 35.90*128*3600, inclusive=True)]

/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.
  


In [13]:
blinkmerged_capital_new

,area_code,link_id,diff,roadname,routenumber,roadtype,link_type_code_x,tollroad,motorway,hwy,...,hwyupdown,mesh_code,road_type_code,lane_num_code,start_trans_id,end_tras_id,link_length,link_type_code_y,start_lon,start_lat
174019,27,83585420,64,首都高速湾岸線,NaN,1,16,1,1,1,...,0,533904.0,40961.0,3.0,4752850.0,4752852.0,352.0,16.0,64336501.0,16306747.0
174020,27,83585485,24,首都高速湾岸線,NaN,1,16,1,1,1,...,0,533904.0,40961.0,3.0,4752852.0,4760412.0,108.0,16.0,64337721.0,16305681.0
174021,27,83585609,48,首都高速湾岸線,NaN,1,16,1,1,1,...,0,533904.0,40961.0,3.0,4752853.0,4752854.0,252.0,16.0,64336863.0,16312838.0
174022,27,83585658,48,首都高速湾岸線,NaN,1,16,1,1,1,...,0,533904.0,40961.0,3.0,4752854.0,4752856.0,253.0,16.0,64336290.0,16311905.0
174023,27,83585806,56,首都高速湾岸線,NaN,1,16,1,1,1,...,0,533904.0,40961.0,3.0,4752858.0,4752859.0,312.0,16.0,64336901.0,16313005.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277763,27,117632419,8,首都高速１号羽田線,NaN,1,16,1,1,1,...,0,533935.0,40961.0,3.0,9795042.0,9795043.0,93.0,16.0,64393660.0,16397423.0
277877,27,118415091,8,首都高速５号池袋線,NaN,1,16,1,1,1,...,0,533955.0,40961.0,3.0,9851262.0,9851263.0,37.0,16.0,64364195.0,16485615.0
277878,27,118415100,56,首都高速５号池袋線,NaN,1,16,1,1,1,...,0,533955.0,40961.0,3.0,9851263.0,6905077.0,349.0,16.0,64364356.0,16485535.0
278692,27,118770411,56,首都高速中央環状線,NaN,1,20,1,1,1,...,0,533936.0,40961.0,4.0,9870120.0,9434909.0,302.0,20.0,64399186.0,16405002.0


# analysis data for longest path
## step0： create connect_dic  {linkid:[linkid1,linkid2]}

In [14]:
connect_dic={}
for linkid in tqdm(blinkmerged_capital_new["link_id"]):
    for _end_crs_id in blinkmerged_capital_new[blinkmerged_capital_new['link_id']==linkid]['end_tras_id'].values:
        for end_link in blinkmerged_capital_new[blinkmerged_capital_new['start_trans_id']==_end_crs_id]['link_id'].values:
            if linkid not in connect_dic.keys():
                connect_dic[linkid]=[end_link]
            else:
                if end_link not in connect_dic[linkid]:
                    connect_dic[linkid].append(end_link)

In [15]:
len(connect_dic.keys())

2589

In [16]:
connect_dic1=deepcopy(connect_dic)
G = nx.DiGraph()
for key in connect_dic1.keys():
    for value in connect_dic1[key]:
        G.add_edge(key, value)

In [17]:
print(G.number_of_nodes())

2841


In [18]:
nodes = list(G.nodes)

## create adjacency_matrix

In [ ]:
#M1 = nx.adjacency_matrix(G,nodelist=nodes)
M1=nx.to_pandas_adjacency(G, nodelist=nodes, dtype=int)

In [ ]:
len(M1)

In [ ]:
#np.savetxt('result/visualization/matrix.txt',M1.toarray(), fmt='%d',)
M1.to_csv('result/metrix/capital_01_relation.csv')

##  create distance adjacency_matrix

In [ ]:
# distance M2
from haversine import haversine, Unit

G2 = nx.DiGraph()
for key in tqdm(connect_dic1.keys()):
    for value in connect_dic1[key]:
        key_latlon=(df[df['link_id']==key]['start_lat'].values[0]/128/3600,df[df['link_id']==key]['start_lon'].values[0]/128/3600)
        value_latlon=(df[df['link_id']==value]['start_lat'].values[0]/128/3600,df[df['link_id']==value]['start_lon'].values[0]/128/3600)
        G2.add_edge(key, value, weight=haversine(key_latlon, value_latlon)*1000)

#df[df['link_id']==83585420]['start_lat'].values[0]/128/3600

In [ ]:
M2=nx.to_pandas_adjacency(G2, nodelist=nodes, dtype=float)

In [ ]:
M2

In [ ]:
for row in tqdm(M2.index):
    for column in M2:
        key_latlon=(df[df['link_id']==row]['start_lat'].values[0]/128/3600,  df[df['link_id']==row]['start_lon'].values[0]/128/3600)
        value_latlon=(df[df['link_id']==column]['start_lat'].values[0]/128/3600,df[df['link_id']==column]['start_lon'].values[0]/128/3600)
        M2.at[row,column]=haversine(key_latlon, value_latlon)*1000

In [ ]:
M2

In [ ]:
M2.to_csv('result/metrix/capital_distance_relation.csv')

## create capital_graph_link_info.csv

In [ ]:
blinkmerged_capital_new_graph= blinkmerged_capital_new.loc[blinkmerged_capital_new["link_id"].isin(nodes)][["link_id","roadname","start_lon","start_lat"]]

In [ ]:
blinkmerged_capital_new_graph=blinkmerged_capital_new_graph.reset_index(drop=True)

In [ ]:
blinkmerged_capital_new_graph=blinkmerged_capital_new_graph.apply({"link_id": lambda x:x,"roadname": lambda x:x, "start_lon": lambda x:x/128/3600, "start_lat": lambda x:x/128/3600})

In [ ]:
blinkmerged_capital_new_graph.to_csv('result/metrix/capital_graph_link_info.csv')